In [1]:
## import libraries

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [3]:
## Load the data

data = pd.read_csv('Churn_Modelling.csv')
print(data.shape)
data.head()

(10000, 14)


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.shape

(10000, 11)

In [5]:
# Label Encoder for Gender
 
label_encoder_gender = LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head(10)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
5,645,Spain,1,44,8,113755.78,2,1,0,149756.71,1
6,822,France,1,50,7,0.00,2,1,1,10062.80,0
7,376,Germany,0,29,4,115046.74,4,1,0,119346.88,1
8,501,France,1,44,4,142051.07,2,0,1,74940.50,0
9,684,France,1,27,2,134603.88,1,1,1,71725.73,0


In [6]:
# One hot encode Geography column

one_hot_encoder_geo = OneHotEncoder()
geo_encoder = one_hot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [7]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns = one_hot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df.shape

(10000, 3)

In [8]:
## Combine the one hot encoded columns with the original data

data = pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [10]:
## Divide the data into X and y

X= data.drop('EstimatedSalary',axis=1)
y = data['EstimatedSalary']

# Split the data in training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [11]:
## save pre-processings encoders 

with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

In [12]:
#Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
##save scaler 
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [14]:
print(X_train.shape)
print(X_test.shape)

(8000, 12)
(2000, 12)


## ANN Regression Implementation

In [17]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

import datetime

In [16]:
## Build the model

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)     # Output layer for regression --> Here default activation function is linear
])


## Compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
## Setup the tensorboard

from tensorflow.keras.callbacks import EarlyStopping,TensorBoard

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir,histogram_freq=1)


In [19]:
## Setup the Early stopping

early_stopping_callbacks = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [20]:
## Train the model

history = model.fit( 
    X_train,y_train,
    validation_data=(X_test,y_test),
    epochs =100,
    callbacks = [tensorflow_callback,early_stopping_callbacks])

Epoch 1/100


250/250 [==============================] - 4s 8ms/step - loss: 100373.3906 - mae: 100373.3906 - val_loss: 98504.1328 - val_mae: 98504.1328
Epoch 2/100
250/250 [==============================] - 1s 5ms/step - loss: 99574.3750 - mae: 99574.3750 - val_loss: 96895.1797 - val_mae: 96895.1797
Epoch 3/100
250/250 [==============================] - 1s 5ms/step - loss: 96747.6406 - mae: 96747.6406 - val_loss: 92764.5391 - val_mae: 92764.5391
Epoch 4/100
250/250 [==============================] - 1s 5ms/step - loss: 91214.5938 - mae: 91214.5938 - val_loss: 85918.2891 - val_mae: 85918.2891
Epoch 5/100
250/250 [==============================] - 1s 5ms/step - loss: 83159.0781 - mae: 83159.0781 - val_loss: 77132.8125 - val_mae: 77132.8125
Epoch 6/100
250/250 [==============================] - 2s 8ms/step - loss: 73851.5781 - mae: 73851.5781 - val_loss: 68168.0547 - val_mae: 68168.0547
Epoch 7/100
250/250 [==============================] - 1s 5ms/step - loss: 65008.1289 - mae: 65008.128

In [24]:
# Save the model

model.save('regression_model.h5')

c:\Users\Dell\python\NLP\ANN-regression\annreg\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs/fit/20250102-201512/

In [23]:
## Evaluate the model on test data

test_loss,test_mae = model.evaluate(X_test,y_test)
print(f"Test loss is: {test_loss} and Test MAE is: {test_mae}.")

63/63 [==============================] - 0s 3ms/step - loss: 50272.9883 - mae: 50272.9883
Test loss is: 50272.98828125 and Test MAE is: 50272.98828125.


# End!